In [1]:
import mne
import os
import scipy.io
import listen_italian_functions
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tnrange, tqdm_notebook

data_path = os.path.dirname(os.path.dirname(os.getcwd()))

subject_name = ['Marco','Martina','Pagani','Pasquale','Sara','Silvia','Silvia2','Tommaso','Alice','Andrea','Daniel',
                'Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu']
#subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
#                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale',
#                'Silvia','Silvia2','Tommaso']


# epoching and saving

In [ ]:
# extract trials of tmax second and remove the wrong answer trials and seperate them in three conditions
Tmin = 0
Tmax = 3.51
trial_len = 2

for s in subject_name:
    raw_fname = data_path + '/python/data/'+s+'_raw.fif'
    raw = mne.io.read_raw_fif(raw_fname,preload=True)
    raw_fname = data_path + '/behaviour/data/subject/'+s+'_behaviour.mat'
    mat = scipy.io.loadmat(raw_fname)
    epochs = listen_italian_functions.epoch(raw, mat,Tmin, Tmax)
    save_path = data_path + '/python/data/coherence_epochs/'+s+'-coh-epo-'+str(Tmin)+'-'+str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs.save(save_path)
    print('----------------------------------------------------------------------------------------------------------------'+s)



# Read the epoches

In [2]:
Tmin = 0
Tmax = 3.51
trial_len = 2

GA_epoches = []
for s in subject_name:
    save_path = data_path + '/python/data/coherence_epochs/'+s+'-coh-epo-'+str(Tmin)+'-' \
    +str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs = mne.read_epochs(save_path)
    GA_epoches.append(epochs)
    print('----------------------------------------------------------------------------------------------------------------'+s)


This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Marco-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Marco-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
114 matching events found
No baseline correction applied
114 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Marco
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Martina-coh-epo-0-3.51-trialLen-2.fif) do

No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Andrea
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Daniel-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Daniel-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
122 matching events found
No baseline correction applied
122 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
--------------------------------------------------------------------------------

Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
110 matching events found
No baseline correction applied
110 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Jonluca
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Laura-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Laura-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
120 matching events found

In [ ]:
condition = ['Hyper','Normal','Hypo']
frames = []
for s in range(0,len(subject_name)):
    df = pd.DataFrame({'Condition':'Hyper','Subject':subject_name[s],'noTrials':GA_epoches[s]['hyper'].get_data().shape[0]},index=[s])
    df = df.append(pd.DataFrame({'Condition':'Normal','Subject':subject_name[s],'noTrials':GA_epoches[s]['normal'].get_data().shape[0]},index=[s]))
    df = df.append(pd.DataFrame({'Condition':'Hypo','Subject':subject_name[s],'noTrials':GA_epoches[s]['hypo'].get_data().shape[0]},index=[s]))
    frames.append(df)
    
data=pd.concat((frames),axis=0)

In [ ]:
a=data.groupby(['Subject','Condition']).sum()['noTrials'].unstack().plot(kind='bar',figsize=(20,5),grid=True)

# Measuring the coherence

In [ ]:
remove_first = 0#seconds

# let's explore some frequency bands
iter_freqs = [
    ('Delta', 1, 3),
    ('Theta', 4, 7),
    ('Alpha', 8, 12),
    ('Beta', 13, 25),
    ('Gamma', 30, 40)
]

iter_freqs = [
    ('fr', 1, 3),
    ('fr', 2, 4),
    ('fr', 3, 5),
    ('fr', 4, 6),
    ('fr', 5, 7),
    ('fr', 6, 8),
    ('fr', 7, 9),
    ('fr', 8, 10),
    ('fr', 9, 11),
    ('fr', 10, 12)
]
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
condition = ['Hyper','Normal','Hypo']
delay = np.arange(0,1.1,0.1)


indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))   

extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'hyper': 1,'normal': 2,'hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
        
for s in tqdm_notebook(range(0,len(subject_name)), desc='Subjects'):
    frames = []
    for d in tqdm_notebook(delay, desc='Delay'):        

        epoch = listen_italian_functions.coherence_preprocess_delay(GA_epoches[s],remove_first,d,trial_len,
                                                                    extra_channels,eeg_channles,info,ch_names,event_id)

        
        for band, fmin, fmax in iter_freqs:
            a = band+ ' '+str(fmin)+' - '+str(fmax)+'Hz'                
            hyper, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['hyper'],fmin, fmax,indices)
            normal, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['normal'],fmin, fmax,indices)
            hypo, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['hypo'],fmin, fmax,indices)
            allC, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch,fmin, fmax,indices)

            hyper = np.split(hyper, len(features))
            normal = np.split(normal, len(features))
            hypo = np.split(hypo, len(features))
            allC = np.split(allC, len(features))

            for f in range(0,len(features)):
                feature = features[f]
                df = pd.DataFrame({'Condition':'Hyper','Freq':a,'Delay':d,'Feature':feature,
                                           'noTrials':GA_epoches[s]['hyper'].get_data().shape[0],
                                           'Subject':subject_name[s],'Data':[hyper[f].flatten()]})
                df = df.append(pd.DataFrame({'Condition':'Normal','Freq':a,'Delay':d,'Feature':feature,
                                             'noTrials':GA_epoches[s]['normal'].get_data().shape[0],
                                             'Subject':subject_name[s],'Data':[normal[f].flatten()]}))
                df = df.append(pd.DataFrame({'Condition':'Hypo','Freq':a,'Delay':d,'Feature':feature,
                                             'noTrials':GA_epoches[s]['hypo'].get_data().shape[0],
                                             'Subject':subject_name[s],'Data':[hypo[f].flatten()]}))
                df = df.append(pd.DataFrame({'Condition':'All','Freq':a,'Delay':d,'Feature':feature,
                                             'noTrials':GA_epoches[s].get_data().shape[0],
                                             'Subject':subject_name[s],'Data':[allC[f].flatten()]}))
                frames.append(df)
        print(d)
        
    data=pd.concat((frames),axis=0)
    save_path = data_path + '/analysis/python/data/coherence-trialLen-'+str(trial_len)+'-removedFirst-'+str(remove_first)+'s-'+subject_name[s]
    data.to_pickle(save_path)

# Measure the coherence in specific delay and shifting the eeg and speech from speech onset

In [ ]:
remove_first = [0,0.1,0.2,0.3,0.4,0.5] #seconds

for rf in remove_first:
        # let's explore some frequency bands
    iter_freqs = [
        ('Delta', 1, 3),
        ('Theta', 4, 7),
        ('Alpha', 8, 12),
        ('Beta', 13, 25),
        ('Gamma', 30, 40)
    ]
    features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
    condition = ['Hyper','Normal','Hypo']
    delay = np.arange(0,1.1,0.1)
    delay = [0,0.1,0.2]
    delay = np.add(delay,0.5) #shift to below algorithm format
    indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))  
    extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
    eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
    event_id = {'hyper': 1,'normal': 2,'hypo': 3}
    ch_types = np.repeat('eeg', len(features)+59)
    ch_names = np.hstack((eeg_channles,features))        
    info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
    ch_names = np.setdiff1d(extra_channels,features)

    for s in tqdm_notebook(range(0,len(subject_name)), desc='Subjects'):
        frames = []
        for d in tqdm_notebook(delay, desc='Delay'):        

            epoch = listen_italian_functions.coherence_preprocess_delay(GA_epoches[s],rf,d,trial_len,
                                                                        extra_channels,eeg_channles,info,ch_names,event_id)


            for band, fmin, fmax in iter_freqs:
                a = band+ ' '+str(fmin)+' - '+str(fmax)+'Hz'                
                hyper, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['hyper'],fmin, fmax,indices)
                normal, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['normal'],fmin, fmax,indices)
                hypo, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['hypo'],fmin, fmax,indices)
                allC, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch,fmin, fmax,indices)

                hyper = np.split(hyper, len(features))
                normal = np.split(normal, len(features))
                hypo = np.split(hypo, len(features))
                allC = np.split(allC, len(features))

                for f in range(0,len(features)):
                    feature = features[f]
                    df = pd.DataFrame({'Condition':'Hyper','Freq':a,'Delay':d,'Feature':feature,
                                               'noTrials':GA_epoches[s]['hyper'].get_data().shape[0],
                                               'Subject':subject_name[s],'Data':[hyper[f].flatten()]})
                    df = df.append(pd.DataFrame({'Condition':'Normal','Freq':a,'Delay':d,'Feature':feature,
                                                 'noTrials':GA_epoches[s]['normal'].get_data().shape[0],
                                                 'Subject':subject_name[s],'Data':[normal[f].flatten()]}))
                    df = df.append(pd.DataFrame({'Condition':'Hypo','Freq':a,'Delay':d,'Feature':feature,
                                                 'noTrials':GA_epoches[s]['hypo'].get_data().shape[0],
                                                 'Subject':subject_name[s],'Data':[hypo[f].flatten()]}))
                    df = df.append(pd.DataFrame({'Condition':'All','Freq':a,'Delay':d,'Feature':feature,
                                                 'noTrials':GA_epoches[s].get_data().shape[0],
                                                 'Subject':subject_name[s],'Data':[allC[f].flatten()]}))
                    frames.append(df)
            print(d)

        data=pd.concat((frames),axis=0)
        save_path = data_path + '/analysis/python/data/shift/coherence-trialLen-'+str(trial_len)+'-removedFirst-'+str(rf)+'s-'+subject_name[s]
        data.to_pickle(save_path)

# Bootstrapping and creating a surrogate distribution for each subject. substract the average of the distribution from the original coherence value.

In [4]:
no_surrogates = 500
remove_first = 0.5 #seconds
iter_freqs = [
    ('fr', 1, 3),
    ('fr', 4, 6)
]
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
delay = np.arange(0,1.1,0.1)

indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))  
extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'hyper': 1,'normal': 2,'hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
    
for s in range(0,len(subject_name)):
    frames = []
    for d in delay:       
        surrogate_coh = listen_italian_functions.coherence_preprocess_delay_surrogate(GA_epoches[s],remove_first,
                                                                                      d,trial_len,extra_channels,
                                                                                      eeg_channles,info,ch_names,
                                                                                      event_id,no_surrogates,
                                                                                      iter_freqs,indices)
        
        # mean or median of the surrogate distribution
        coh=surrogate_coh.mean(axis=2)
        
        for fr in range(0,len(iter_freqs)):
            a = str(iter_freqs[fr][0])+ ' '+str(iter_freqs[fr][1])+' - '+str(iter_freqs[fr][2])+'Hz'
            cc = np.split(coh[:,fr], len(features))
            for f in range(0,len(features)):
                feature = features[f]
                df = pd.DataFrame({'Condition':'All','Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                   'Surrogate':[cc[f].flatten()]})
                frames.append(df)                
    data=pd.concat((frames),axis=0)
    save_path = data_path + '/python/data/SurrogateCoherence/SurrogateCoherence-trialLen-'+str(trial_len)+'-removedFirst-'\
    +str(remove_first)+'s-'+subject_name[s]
    data.to_pickle(save_path)     
        
    

--------------------7


KeyboardInterrupt: 

In [ ]:
# putit into one file
save_path = data_path + '/python/data/SurrogateCoherence/SurrogateCoherence-trialLen-'+str(trial_len)+'-removedFirst-'\
    +str(remove_first)+'s'

frames = []
for s in subject_name:    
    frames.append(pd.read_pickle(save_path+'-'+s))
    
data = pd.concat((frames),axis=0)